In [1]:
from lsst.ts import salobj
import asyncio
import numpy as np

import SALPY_ATPtg
import SALPY_ATMCS
import SALPY_ATPneumatics
import SALPY_ATHexapod
import SALPY_ATAOS
import SALPY_ATDome
import SALPY_ATDomeTrajectory

In [8]:
r1 = salobj.Remote(SALPY_ATPneumatics)

r2 = salobj.Remote(SALPY_ATAOS)

r3 = salobj.Remote(SALPY_ATHexapod)

r4 = salobj.Remote(SALPY_ATMCS)

r5 = salobj.Remote(SALPY_ATPtg)

r6 = salobj.Remote(SALPY_ATDome, 1)

r7 = salobj.Remote(SALPY_ATDomeTrajectory)

In [9]:
def get_callback_for(name):
    def print_pressure_callback(id_data):
        print(f'{name} {id_data.pressure}')
    return print_pressure_callback

def print_evt_hexapodCorrectionStarted_callback(id_data):
    print('CorrectionStarted:',id_data.azimuth, id_data.elevation, 
          id_data.hexapod_x, 
          id_data.hexapod_y, 
          id_data.hexapod_z, 
          id_data.hexapod_u, 
          id_data.hexapod_v, 
          id_data.hexapod_w)

def print__moveToPosition_callback(id_data):
    print('-->', 
          id_data.positionX, 
          id_data.positionY, 
          id_data.positionZ, 
          id_data.positionU, 
          id_data.positionV, 
          id_data.positionW)

def get_print_log_message_callback_for(name):
    def print_log_message(id_data):
        print(f"----\n {name}:\n LOG:{id_data.message}\n TRACE:{id_data.traceback}\n----" )

    return print_log_message

def print_detailed_state(id_data):
    print(id_data.substate)

def get_error_code_callback_for(name):
    def print_error_code(id_data):
        print(f'{name}::{id_data.errorCode}::{id_data.errorReport}')
    return print_error_code

def get_ss_callback_for(name):
    def print_summary_state(id_data):
        print(f"{name}: {salobj.State(id_data.summaryState)!r}")
    return print_summary_state
    

In [ ]:
r1.evt_m1SetPressure.callback = get_callback_for("ATPneu:M1")
r1.evt_m2SetPressure.callback = get_callback_for("ATPneu:M2")
r2.evt_m1CorrectionStarted.callback = get_callback_for("ATAOS:M1:")
r2.evt_m2CorrectionStarted.callback = get_callback_for("ATAOS:M2")
r3.evt_positionUpdate.callback = print__moveToPosition_callback
r2.evt_hexapodCorrectionStarted.callback = print_evt_hexapodCorrectionStarted_callback

In [ ]:
for r in [r1, r2, r3, r4, r5, r6, r7]:
    r.evt_logMessage.callback = get_print_log_message_callback_for(r.salinfo.name)
    r.evt_summaryState.callback = get_ss_callback_for(r.salinfo.name)
    r.evt_errorCode.callback = get_error_code_callback_for(r.salinfo.name)

In [ ]:
asyncio.get_event_loop().run_forever()

In [ ]:
r1.salinfo.name

In [16]:
for r in [r1, r2, r3, r4, r5, r6, r7]:
    state = salobj.State(r.evt_summaryState.get().summaryState)
    print(f"{r.salinfo.name}::{state!r}")
#     await salobj.set_summary_state(r, salobj.State.STANDBY, timeout=300)

ATPneumatics::<State.ENABLED: 2>
ATAOS::<State.ENABLED: 2>
ATHexapod::<State.ENABLED: 2>
ATMCS::<State.ENABLED: 2>
ATPtg::<State.ENABLED: 2>
ATDome::<State.STANDBY: 5>
ATDomeTrajectory::<State.STANDBY: 5>


In [11]:
await salobj.set_summary_state(r1, salobj.State.ENABLED, timeout=300)

In [12]:
await salobj.set_summary_state(r2, salobj.State.ENABLED, settingsToApply='test')

In [13]:
await salobj.set_summary_state(r3, salobj.State.ENABLED, timeout=300,
                               settingsToApply='Default1')

In [14]:
await salobj.set_summary_state(r4, salobj.State.ENABLED, timeout=300)

In [15]:
await salobj.set_summary_state(r5, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(r6, salobj.State.ENABLED, settingsToApply='test')

In [ ]:
await salobj.set_summary_state(r7, salobj.State.ENABLED)

In [ ]:
await asyncio.sleep(.5)
print("*** running")
try:
    await r2.cmd_applyCorrection.start()
finally:
    print("*** done")
    await asyncio.sleep(.5)

In [ ]:
r2.cmd_enableCorrection.set(enableAll=True)

In [ ]:
await r2.cmd_enableCorrection.start()

In [ ]:
r3.evt_logMessage.callback = None

In [ ]:
await asyncio.sleep(5.)

In [ ]:
r2.cmd_disableCorrection.set(disableAll=True)

In [ ]:
await r2.cmd_disableCorrection.start()

In [ ]:
r2.cmd_setLogLevel.set(level=10)
await r2.cmd_setLogLevel.start()

In [ ]:
r2.evt_logMessage.callback = get_print_log_message_callback_for(r2.salinfo.name)
r2.evt_errorCode.callback = get_error_code_callback_for(r2.salinfo.name)

In [19]:
await r1.cmd_openMasterAirSupply.start()

CommandIdAck(cmd_id=1559745061, ack=(ack=303, error=0, result='Ok'))

In [20]:
await r1.cmd_openInstrumentAirValve.start()

CommandIdAck(cmd_id=1559745061, ack=(ack=303, error=0, result='Ok'))

In [17]:
await r1.cmd_m1OpenAirValve.start()

CommandIdAck(cmd_id=1559745061, ack=(ack=303, error=0, result='Ok'))

In [18]:
await r1.cmd_m2OpenAirValve.start()

CommandIdAck(cmd_id=1559745061, ack=(ack=303, error=0, result='Ok'))

In [ ]:
from astropy import units as u

In [ ]:
pressure = 5.*u.imperial.psi

In [ ]:
r1.cmd_m1SetPressure.set(pressure=45117.99254084)

In [ ]:
await r1.cmd_m1SetPressure.start()

In [ ]:
r3.cmd_moveToPosition.set(z=6.869660377502441)

In [ ]:
await r3.cmd_moveToPosition.start()

In [ ]:
await r1.cmd_m2SetPressure.start()

In [ ]:
await salobj.set_summary_state(r2, salobj.State.STANDBY)

In [ ]:
await r2.cmd_exitControl.start()

In [ ]:
await r2.evt_heartbeat.next(flush=True)

In [ ]:
await salobj.set_summary_state(r3, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(r5, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(r6, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(r7, salobj.State.STANDBY)

In [ ]:
import palpy

In [2]:
names = ["a", "b", "c"]
values = [0, 0, 0]

In [6]:
zip(names, False)

TypeError: zip argument #2 must support iteration